In [37]:
import kaolin
import torch
import math
import matplotlib
import os
import numpy as np
import matplotlib.pyplot as plt
import polyscope as ps
import trimesh
import diffvoronoi

from scipy.spatial import Voronoi, voronoi_plot_2d, Delaunay
from io import BytesIO
from PIL import Image
import sdfpred_utils.sdfpred_utils as su
import sdfpred_utils.sdf_MLP as mlp
import sdfpred_utils.sdf_functions as sdf
import sdfpred_utils.loss_functions as lf
import sdfpred_utils.Steik_data3d as sd3d
import sdfpred_utils.Steik_Loss as sl
import sdfpred_utils.Steik_utils as Stu 
import open3d as o3d

#cuda devices
device = torch.device("cuda:0")
print("Using device: ", torch.cuda.get_device_name(device))

#default tensor types
#torch.set_default_tensor_type(torch.cuda.DoubleTensor)

multires = 2
input_dims = 3
lr_sites = 0.005/2
lr_model = 0.00005*2
destination = "./images/autograd/HotSpot/"
mesh = ["gargoyle","/home/wylliam/dev/Kyushu_experiments/data/gargoyle"]
trained_model_path = "/home/wylliam/dev/HotSpot/log/3D/pc/HotSpot-all-2025-04-24-18-16-03/gargoyle/gargoyle/trained_models/model.pth"


Using device:  NVIDIA GeForce RTX 3090


In [38]:
num_centroids = 32**3

print("Creating new sites")
noise_scale = 0.1
domain_limit = 1
x = torch.linspace(-domain_limit, domain_limit, int(round(num_centroids**(1/3))))
y = torch.linspace(-domain_limit, domain_limit, int(round(num_centroids**(1/3))))
z = torch.linspace(-domain_limit, domain_limit, int(round(num_centroids**(1/3))))
meshgrid = torch.meshgrid(x, y, z)
meshgrid = torch.stack(meshgrid, dim=3).view(-1, 3)
#add noise to meshgrid
meshgrid += torch.randn_like(meshgrid) * noise_scale
sites = meshgrid.to(device, dtype=torch.float32).requires_grad_(True)

print("Sites shape: ", sites.shape)
ps.init()
ps.register_point_cloud("initial_cvt_grid",sites.detach().cpu().numpy())

# # load pointcloud used for sdf training
# pointcloud = o3d.io.read_point_cloud(mesh[1]+".ply")
# print("Pointcloud shape: ", np.asarray(pointcloud.points).shape)
# # sample pointcloud to 150*32*32
# chamfer_distance_pc_gt = pointcloud.uniform_down_sample(int((128**3)/(150*32*32)))
# chamfer_distance_pc_gt = np.asarray(chamfer_distance_pc_gt.points)
# print("Chamfer distance pointcloud shape: ", chamfer_distance_pc_gt.shape)


# ps.register_point_cloud("pointcloud_gt", chamfer_distance_pc_gt)
# chamfer_distance_pc_gt = torch.tensor(chamfer_distance_pc_gt, device=device, dtype=torch.float32)

Creating new sites
Sites shape:  torch.Size([32768, 3])


In [39]:
#LOAD MODEL WITH HOTSPOT
import sys
sys.path.append("3rdparty/HotSpot")
from dataset import shape_3d
import models.Net as Net

loss_type = "igr_w_heat"
loss_weights = [350, 0, 0, 1, 0, 0, 20]

train_set = shape_3d.ReconDataset(
    file_path = mesh[1]+".ply",
    n_points=32*32*150,#15000, #args.n_points,
    n_samples=10001, #args.n_iterations,
    grid_res=256, #args.grid_res,
    grid_range=1.1, #args.grid_range,
    sample_type="uniform_central_gaussian", #args.nonmnfld_sample_type,
    sampling_std=0.5, #args.nonmnfld_sample_std,
    n_random_samples=7500, #args.n_random_samples,
    resample=True,
    compute_sal_dist_gt=(
        True if "sal" in loss_type and loss_weights[5] > 0 else False
    ),
    scale_method="mean"#"mean" #args.pcd_scale_method,
)

model = Net.Network(
    latent_size=0,#args.latent_size,
    in_dim=3,
    decoder_hidden_dim=128,#args.decoder_hidden_dim,
    nl="sine",#args.nl,
    encoder_type="none",#args.encoder_type,
    decoder_n_hidden_layers=5,#args.decoder_n_hidden_layers,
    neuron_type="quadratic",#args.neuron_type,
    init_type="mfgi",#args.init_type,
    sphere_init_params=[1.6, 0.1],#args.sphere_init_params,
    n_repeat_period=30#args.n_repeat_period,
)
model.to(device)

######       
test_dataloader = torch.utils.data.DataLoader(train_set, batch_size=1, shuffle=False, num_workers=0, pin_memory=False)   
test_data = next(iter(test_dataloader))
mnfld_points = test_data["mnfld_points"].to(device)
mnfld_points.requires_grad_()
print("mnfld_points shape: ", mnfld_points.shape)
if torch.cuda.is_available():
    map_location = torch.device("cuda")
else:
    map_location = torch.device("cpu")
model.load_state_dict(torch.load(trained_model_path, weights_only=True, map_location=map_location))

mnfld_points shape:  torch.Size([1, 153600, 3])


<All keys matched successfully>

In [40]:
#add mnfld points with random noise to sites 

sites = torch.cat((sites, mnfld_points.squeeze(0)+(torch.rand_like(mnfld_points.squeeze(0))-0.5)*0.1), dim=0)
# make sites a leaf tensor
sites = sites.detach().requires_grad_()

sites_pred = model(sites)#["nonmanifold_pnts_pred"]
mnfld_preds = model(mnfld_points)#["nonmanifold_pnts_pred"]

ps_cloud = ps.register_point_cloud("initial_cvt_grid+pc_gt",sites.detach().cpu().numpy())
mnf_cloud = ps.register_point_cloud("mnfld_points_pred",mnfld_points.squeeze(0).detach().cpu().numpy())
mnf_cloud.add_scalar_quantity("mnfld_points_pred", mnfld_preds.reshape(-1).detach().cpu().numpy(), enabled=True)
ps_cloud.add_scalar_quantity("vis_grid_pred", sites_pred.reshape(-1).detach().cpu().numpy(), enabled=True)

initial_mesh = su.get_zero_crossing_mesh_3d(sites, model)
ps.register_surface_mesh("initial Zero-Crossing faces", initial_mesh[0], initial_mesh[1])

ps.show()

In [41]:
# SITES OPTIMISATION LOOP
cvt_loss_values = []
min_distance_loss_values = []
chamfer_distance_loss_values = []
eikonal_loss_values = []
domain_restriction_loss_values = []
sdf_loss_values = []
div_loss_values = []
loss_values = []



def autograd(sites, model, max_iter=100, stop_train_threshold=1e-6, upsampling=0, lambda_weights = [0.1,1.0,0.1,0.1,1.0,1.0,0.1]):
    optimizer = torch.optim.Adam([
    {'params': [p for _, p in model.named_parameters()], 'lr': 5e-5},
    {'params': [sites], 'lr': lr_sites}
], betas=(0.9, 0.999))

    prev_loss = float("inf")
    best_loss = float("inf")
    upsampled = 0.0
    epoch = 0
    lambda_cvt = lambda_weights[0]
    # lambda_pc = lambda_weights[1]
    lambda_min_distance = lambda_weights[2]
    # lambda_laplace = lambda_weights[3]
    lamda_chamfer = lambda_weights[4]
    lambda_eikonal = lambda_weights[5]
    lambda_domain_restriction = lambda_weights[6]
    # lambda_target_points = lambda_weights[7]
    lambda_sdf = 5e3
    lambda_div = 1e2
    lambda_eikonal = 5e1
    best_sites = sites.clone()
    best_sites.best_loss = best_loss
    
    while epoch <= max_iter:
        optimizer.zero_grad()
        
        # compute voronoi and delaunay once for each epoch and pass it around
        # Compute Voronoi diagram
        sites_np = sites.detach().cpu().numpy()
        #vor = Voronoi(sites_np)
        #tri = Delaunay(sites_np)
      
      
        d3dsimplices = diffvoronoi.get_delaunay_simplices(sites_np.reshape(input_dims*sites_np.shape[0]))
        d3dsimplices = np.array(d3dsimplices)
        #print("Delaunay simplices shape: ", d3dsimplices.shape)
        
        vertices_to_compute, bisectors_to_compute = su.compute_zero_crossing_vertices_3d(sites, None, None, d3dsimplices, model)
                
        #vertices_to_compute, bisectors_to_compute = su.compute_zero_crossing_vertices_3d(sites, None, tri, None, model)
        #vertices_to_compute, bisectors_to_compute = su.compute_zero_crossing_vertices_3d(sites, vor, tri, None, model)
        
        vertices = su.compute_vertices_3d_vectorized(sites, vertices_to_compute)    
        bisectors = su.compute_all_bisectors_vectorized(sites, bisectors_to_compute)
        points = torch.cat((vertices, bisectors), 0)
        print("points", points.shape) 

        # Compute losses       
        #cvt_loss = lf.compute_cvt_loss_vectorized_voronoi(sites, vor)
        #cvt_loss = lf.compute_cvt_loss_vectorized_delaunay(sites, tri)
    
        cvt_loss = lf.compute_cvt_loss_vectorized_delaunay(sites, None, d3dsimplices)
        print("CVT loss: ", cvt_loss)
        #min_distance_loss = lf.sdf_weighted_min_distance_loss(model, sites)
        
        from pytorch3d.loss import chamfer_distance
        chamfer_loss, _ = chamfer_distance(mnfld_points.detach(), points.unsqueeze(0))
        print(f"After Chamfer loss PYTORCH3D {chamfer_loss} : Allocated: {torch.cuda.memory_allocated() / 1e6} MB, Reserved: {torch.cuda.memory_reserved() / 1e6} MB")
            
        
        # chamfer_loss = lf.chamfer_distance(mnfld_points, points)
        
        sites_loss = (
            lambda_cvt * cvt_loss +
            lamda_chamfer * chamfer_loss
        )
            
        loss = sites_loss
        loss_values.append(loss.item())
        print(f"Epoch {epoch}: loss = {loss.item()}")
        print(f"before loss.backward(): Allocated: {torch.cuda.memory_allocated() / 1e6} MB, Reserved: {torch.cuda.memory_reserved() / 1e6} MB")

        loss.backward()
        print(f"After loss.backward(): Allocated: {torch.cuda.memory_allocated() / 1e6} MB, Reserved: {torch.cuda.memory_reserved() / 1e6} MB")
        print("-----------------")
        
        optimizer.step()
        
        if loss.item() < best_loss:
            best_loss = loss.item()
            best_epoch = epoch
            best_sites = sites.clone()
            best_sites.best_loss = best_loss
            #if upsampled > 0:
                #print(f"UPSAMPLED {upsampled} Best Epoch {best_epoch}: Best loss = {best_loss}")
                #return best_sites
        
        if abs(prev_loss - loss.item()) < stop_train_threshold:
            print(f"Converged at epoch {epoch} with loss {loss.item()}")
            #break
        
        prev_loss = loss.item() 
        
        # if epoch>100 and (epoch // 100) == upsampled+1 and loss.item() < 0.5 and upsampled < upsampling:
        if epoch/max_iter > (upsampled+1)/(upsampling+1) and upsampled < upsampling:
            print("sites length BEFORE UPSAMPLING: ",len(sites))
            sites = su.upsampling_vectorized(sites, tri=None, vor=None, simplices=d3dsimplices, model=model)
            sites = sites.detach().requires_grad_(True)
            optimizer = torch.optim.Adam([{'params': [p for _, p in model.named_parameters()], 'lr': lr_model},
                                          {'params': [sites], 'lr': lr_sites}])
            upsampled += 1.0
            print("sites length AFTER: ",len(sites))
            
          
        if epoch % (max_iter/50) == 0:
            #print(f"Epoch {epoch}: loss = {loss.item()}")
            #print(f"Best Epoch {best_epoch}: Best loss = {best_loss}")
            #save model and sites
            site_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_sites_{num_centroids}_chamfer{lamda_chamfer}.pth'
            model_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_model_{num_centroids}_chamfer{lamda_chamfer}.pth'
            torch.save(model.state_dict(), model_file_path)
            torch.save(sites, site_file_path)
            
        epoch += 1           
    
    #Export the sites, their sdf values, the gradients of the sdf values and the hessian
    sdf_values = model(sites)

    sdf_gradients = torch.autograd.grad(outputs=sdf_values, inputs=sites, grad_outputs=torch.ones_like(sdf_values), create_graph=True, retain_graph=True,)[0] # (N, 3)

    N, D = sites.shape
    hess_sdf = torch.zeros(N, D, D, device=sites.device)
    for i in range(D):
        grad2 = torch.autograd.grad(outputs=sdf_gradients[:, i], inputs=sites, grad_outputs=torch.ones_like(sdf_gradients[:, i]), create_graph=False, retain_graph=True,)[0] # (N, 3)
        hess_sdf[:, i, :] = grad2 # fill row i of each 3×3 Hessian
    
    np.savez(f'{mesh[0]}_to_clip.npz', sites=sites.detach().cpu().numpy(), sdf_values=sdf_values.detach().cpu().numpy(), sdf_gradients=sdf_gradients.detach().cpu().numpy(), sdf_hessians=hess_sdf.detach().cpu().numpy())

    return best_sites

In [42]:
#lambda_weights = [252,0,0,0,10.211111,0,100,0]
lambda_weights = [100,0,0,0,1000,0,100,0]


lambda_cvt = lambda_weights[0]
lambda_sdf = lambda_weights[1]
lambda_min_distance = lambda_weights[2]
lambda_laplace = lambda_weights[3]
lamda_chamfer = lambda_weights[4]
lambda_eikonal = lambda_weights[5]
lambda_domain_restriction = lambda_weights[6]
lambda_true_points = lambda_weights[7]

max_iter = 100

In [43]:
site_file_path = f'{destination}{max_iter}_cvt_{lambda_cvt}_chamfer_{lamda_chamfer}_eikonal_{lambda_eikonal}.npy'
#check if optimized sites file exists
if not os.path.exists(site_file_path):
    #import sites
    print("Importing sites")
    sites = np.load(site_file_path)
    sites = torch.from_numpy(sites).to(device).requires_grad_(True)
else:
    # import cProfile, pstats
    # import time
    # profiler = cProfile.Profile()
    # profiler.enable()

#     with torch.profiler.profile(activities=[
#             torch.profiler.ProfilerActivity.CPU,
#             torch.profiler.ProfilerActivity.CUDA,
#         ],
#         record_shapes=False,
#         with_stack=True  # Captures function calls
#     ) as prof:
#         sites = autograd(sites, model, max_iter=max_iter, upsampling=0, lambda_weights=lambda_weights)
#         torch.cuda.synchronize()
# # 
#     print(prof.key_averages().table(sort_by="self_cuda_time_total"))
#     prof.export_chrome_trace("trace.json")
    
    # 
    sites = autograd(sites, model, max_iter=max_iter, upsampling=0, lambda_weights=lambda_weights)

    
    # profiler.disable()
    # stats = pstats.Stats(profiler).sort_stats('cumtime')
    # stats.print_stats()
    # stats.dump_stats(f'{destination}{mesh[0]}{max_iter}_3d_profile_{num_centroids}_chamfer{lamda_chamfer}.prof')
    
    
    sites_np = sites.detach().cpu().numpy()
    np.save(site_file_path, sites_np)
    
    

print("Sites length: ", len(sites))
print("min sites: ", torch.min(sites))
print("max sites: ", torch.max(sites))
#ps_cloud = ps.register_point_cloud("best_optimized_cvt_grid",sites.detach().cpu().numpy())
    
lim=torch.abs(torch.max(sites)).detach().cpu().numpy()*1.1
#plot_voronoi_3d(sites,lim,lim,lim)

points torch.Size([307157, 3])
CVT loss:  tensor(0.0016, device='cuda:0', grad_fn=<MeanBackward0>)
After Chamfer loss PYTORCH3D 4.0332721255254e-05 : Allocated: 7037.378048 MB, Reserved: 15267.26656 MB
Epoch 0: loss = 0.20451991260051727
before loss.backward(): Allocated: 7037.37856 MB, Reserved: 15267.26656 MB
After loss.backward(): Allocated: 6911.032832 MB, Reserved: 15267.26656 MB
-----------------
points torch.Size([296504, 3])
CVT loss:  tensor(0.0024, device='cuda:0', grad_fn=<MeanBackward0>)
After Chamfer loss PYTORCH3D 2.269097967655398e-05 : Allocated: 7036.62336 MB, Reserved: 15267.26656 MB
Epoch 1: loss = 0.2583116590976715
before loss.backward(): Allocated: 7036.62336 MB, Reserved: 15267.26656 MB
After loss.backward(): Allocated: 6915.184128 MB, Reserved: 15267.26656 MB
-----------------
points torch.Size([298571, 3])
CVT loss:  tensor(0.0017, device='cuda:0', grad_fn=<MeanBackward0>)
After Chamfer loss PYTORCH3D 1.7870372175821103e-05 : Allocated: 7036.694528 MB, Reserved

In [44]:
epoch = 100

model_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_model_{num_centroids}_chamfer{lamda_chamfer}.pth'
site_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_sites_{num_centroids}_chamfer{lamda_chamfer}.pth'
 
sites = torch.load(site_file_path)

sites_np = sites.detach().cpu().numpy()
model.load_state_dict(torch.load(model_file_path))
#
#polyscope_sdf(model)
#
print("model", model_file_path)
print("sites", site_file_path)
ps_cloud = ps.register_point_cloud(f"{epoch} epoch_cvt_grid",sites_np)



model ./images/autograd/HotSpot/gargoyle100_100_3d_model_32768_chamfer1000.pth
sites ./images/autograd/HotSpot/gargoyle100_100_3d_sites_32768_chamfer1000.pth


In [45]:
final_mesh = su.get_zero_crossing_mesh_3d(sites, model)
ps.register_surface_mesh("Zero-Crossing faces final", final_mesh[0], final_mesh[1])

import scipy.spatial as spatial
from scipy.spatial import Delaunay

tri = Delaunay(sites_np)
delaunay_vertices =torch.tensor(np.array(tri.simplices), device=device)
sdf_values = model(sites)


# Assuming sites is a PyTorch tensor of shape [M, 3]
sites = sites.unsqueeze(0)  # Now shape [1, M, 3]

# Assuming SDF_Values is a PyTorch tensor of shape [M]
sdf_values = sdf_values.unsqueeze(0)  # Now shape [1, M]

marching_tetrehedra_mesh = kaolin.ops.conversions.marching_tetrahedra(sites, delaunay_vertices, sdf_values, return_tet_idx=False)
print(marching_tetrehedra_mesh)
vertices_list, faces_list = marching_tetrehedra_mesh
vertices = vertices_list[0]
faces = faces_list[0]
vertices_np = vertices.detach().cpu().numpy()  # Shape [N, 3]
faces_np = faces.detach().cpu().numpy()  # Shape [M, 3] (triangles)
ps.register_surface_mesh("Marching Tetrahedra Mesh final", vertices_np, faces_np)


ps.show()

[(tensor([[-0.3156, -0.0883, -0.2731],
        [-0.3131, -0.0829, -0.2821],
        [-0.3166, -0.0795, -0.2744],
        ...,
        [ 0.2041, -0.0145, -0.0040],
        [ 0.3079, -0.1212, -0.2902],
        [-0.1589, -0.1193,  0.3620]], device='cuda:0', grad_fn=<SumBackward1>),), (tensor([[35080,  5432, 54927],
        [17121, 62631,  4579],
        [31444, 68665, 56083],
        ...,
        [ 7913,  7915, 64509],
        [58882, 46005, 64510],
        [58882, 46003, 46005]], device='cuda:0'),)]


In [46]:
#plot loss